In [13]:
#Importing Libraries

import pandas as pd
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import warnings
import os
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [14]:
#Setting up the credentials por the Spotify API 
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [16]:
#Function to get the artist id from the artist name
def get_artist_id(artist):
    try:
        artist_results = sp.search(q='artist:' + artist, type='artist')
        return artist_results['artists']['items'][0]['id']
    except:
        return None
    
#Function to get the track id from all the artist tracks
def get_artist_tracks(artist_id):
    try:
        tracks = sp.artist_top_tracks(artist_id)
        return [track['id'] for track in tracks['tracks']]
    except:
        return None
    
#Function to get the playlist id from the playlist name
def get_playlist_id(playlist_name):
    try:
        playlist_results = sp.search(q=playlist_name, type='playlist')
        return playlist_results['playlists']['items'][0]['id']
    except:
        return None
    
#Function to get the track ID from the playlist
def get_playlist_tracks(playlist_id):
    try:
        playlist = sp.playlist_tracks(playlist_id)
        return [track['track']['id'] for track in playlist['items']]
    except:
        return None
    
#Function to get the name, artist and audio features from the track
def get_track_features(track_id):
    try:
        track = sp.track(track_id)
        features = sp.audio_features(track_id)
        return {
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'acousticness': features[0]['acousticness'],
            'analysis_url': features[0]['analysis_url'],
            'danceability': features[0]['danceability'],
            'duration_ms': features[0]['duration_ms'],
            'energy': features[0]['energy'],
            'id': features[0]['id'],
            'instrumentalness': features[0]['instrumentalness'],
            'key': features[0]['key'],
            'liveness': features[0]['liveness'],
            'loudness': features[0]['loudness'],
            'mode': features[0]['mode'],
            'speechiness': features[0]['speechiness'],
            'tempo': features[0]['tempo'],
            'time_signature': features[0]['time_signature'],
            'track_href': features[0]['track_href'],
            'type': features[0]['type'],
            'uri': features[0]['uri'],
            'valence': features[0]['valence']
        }
    except:
        return None
    
#Function to get the albums from an artist
def get_artist_albums(artist_id):
    try:
        albums = sp.artist_albums(artist_id)
        return [album['id'] for album in albums['items']]
    except:
        return None
    
#Function to get the tracks from an album
def get_album_tracks(album_id):
    try:
        tracks = sp.album_tracks(album_id)
        return [track['id'] for track in tracks['items']]
    except:
        return None

In [17]:
#Getting the track features of the top 50 global playlist
Playlist = 'Top 50 - Global'
playlist_id = get_playlist_id(Playlist)
playlist_tracks = get_playlist_tracks(playlist_id)


audio_features = []
for track in playlist_tracks:
    audio_features.append(get_track_features(track))

In [18]:
#Creating a dataframe
track_data = pd.DataFrame(audio_features)

In [19]:
#Checking the data
track_data.head()

,name,artist,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,we can't be friends (wait for your love),Ariana Grande,0.065700,https://api.spotify.com/v1/audio-analysis/51ZQ...,0.645,228639,0.663,51ZQ1vr10ffzbwIjDCwqm4,0.000032,5,0.0751,-8.305,1,0.0447,115.830,4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,audio_features,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,0.287
1,End of Beginning,Djo,0.035100,https://api.spotify.com/v1/audio-analysis/3qhl...,0.689,159246,0.454,3qhlB30KknSejmIvZZLjOD,0.002590,2,0.0707,-7.643,1,0.0584,159.982,4,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,audio_features,spotify:track:3qhlB30KknSejmIvZZLjOD,0.912
2,Beautiful Things,Benson Boone,0.151000,https://api.spotify.com/v1/audio-analysis/6tNQ...,0.472,180304,0.471,6tNQ70jh4OwmPGpYy6R2o9,0.000000,10,0.1400,-5.692,1,0.0603,105.029,3,https://api.spotify.com/v1/tracks/6tNQ70jh4Owm...,audio_features,spotify:track:6tNQ70jh4OwmPGpYy6R2o9,0.219
3,Like That,Future,0.007090,https://api.spotify.com/v1/audio-analysis/2tud...,0.814,267707,0.676,2tudvzsrR56uom6smgOcSf,0.000013,11,0.1190,-4.670,0,0.2310,162.012,4,https://api.spotify.com/v1/tracks/2tudvzsrR56u...,audio_features,spotify:track:2tudvzsrR56uom6smgOcSf,0.312
4,i like the way you kiss me,Artemas,0.000938,https://api.spotify.com/v1/audio-analysis/2Gxr...,0.599,142515,0.946,2GxrNKugF82CnoRFbQfzPf,0.010600,11,0.0826,-4.263,1,0.0447,151.647,4,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,audio_features,spotify:track:2GxrNKugF82CnoRFbQfzPf,0.747


In [9]:
#Creating a loop to get the track features from an album for each unique artist in the playlist, waiting between each request
artist_data = track_data['artist'].unique()
album_tracks = []
start = 0
for stop in range(0, len(artist_data) + 100, 100):
    if start != stop:
        print(start, stop)
        artist_id = get_artist_id(artist_data)
        artist_albums = get_artist_albums(artist_id)
        for album in artist_albums:
            album_tracks.extend(get_album_tracks(album))
        time.sleep(45)
        start = stop


0 100


In [10]:
album_tracks

['7xTbVQSuLpM1SRIKMcaPCI',
 '7aOgyKhHxAhrzTKkLGF3Ha',
 '3K6TfJcMDBhZmYrK2oI8If',
 '0go0Gvkh1a57FMud7h1SXd',
 '4llxVu7gt1uG1zN8EcWwi3',
 '30eYVvVYrXwVKNCyZ9HpPg',
 '72bYIhu9vxqA14xM3j7Max',
 '19nGr45kS0vRbuWeEnarRW',
 '5Hqo6dHSTPilabNfzZ3MyP',
 '3nH6JdYvrM7cnLT7xMrEKX',
 '6UUmCyNYOBq4fxfbN1sVrK',
 '6GnvnyxQ0ZnCTHFyvrdMhI',
 '1f8raciYdYBqsjv5v21S53',
 '3iJOkxYP59kFJ5OLgHo4Tp',
 '0blfewz7XXrjsZd5WnHwgE',
 '4RSoXPyBHKolShYA6FhvfP',
 '1UjPANunXLp0G9hbQNyPh9',
 '2o1pb13quMReXZqE7jWsgq',
 '1Rweq7vNjK4kZBbGrmxQsl',
 '0o3ua5ufFK7nfRzbDUNoGA',
 '13jRSCLoWbCoU1XEdJRbgn',
 '3RwWW7KeVhHGayYJgUL5eZ',
 '142PiXzA84lmEw2RstFHFa',
 '14kyXBpg91RVq8bNRDS1q2',
 '0Lmbke3KNVFXtoH2mMSHCw',
 '5D34wRmbFS29AjtTOP2QJe',
 '51ZQ1vr10ffzbwIjDCwqm4',
 '5cQX9BexkPIEESTZijJhXg',
 '6XXKB32Om6WuXg3uEWwTob',
 '4mHM5d2fK3n8xgymjf92w2',
 '5UkqH9t76GxI48YHA0thsy',
 '6qT31f20CXWq0zkz4BME32',
 '5aKqhRdBmC5Qp2c3oek9ft',
 '4HBDccZgvC769vqQ2QHjji',
 '4EjLfjUNonstEf9ZED4gce',
 '1WPjEBaYjOh65gQF7EwwOp',
 '2tQew7pG4IhgNGplOSNIj7',
 

In [20]:
#Adding the album tracks to the dataframe
album_audio_features = []
for track in album_tracks:
    album_audio_features.append(get_track_features(track))
    
album_data = pd.DataFrame(album_audio_features)
track_data = pd.concat([track_data, album_data], axis=0)
track_data.reset_index(drop=True, inplace=True)

#Checking the data
track_data.head()

,name,artist,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,we can't be friends (wait for your love),Ariana Grande,0.065700,https://api.spotify.com/v1/audio-analysis/51ZQ...,0.645,228639,0.663,51ZQ1vr10ffzbwIjDCwqm4,0.000032,5,0.0751,-8.305,1,0.0447,115.830,4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,audio_features,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,0.287
1,End of Beginning,Djo,0.035100,https://api.spotify.com/v1/audio-analysis/3qhl...,0.689,159246,0.454,3qhlB30KknSejmIvZZLjOD,0.002590,2,0.0707,-7.643,1,0.0584,159.982,4,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,audio_features,spotify:track:3qhlB30KknSejmIvZZLjOD,0.912
2,Beautiful Things,Benson Boone,0.151000,https://api.spotify.com/v1/audio-analysis/6tNQ...,0.472,180304,0.471,6tNQ70jh4OwmPGpYy6R2o9,0.000000,10,0.1400,-5.692,1,0.0603,105.029,3,https://api.spotify.com/v1/tracks/6tNQ70jh4Owm...,audio_features,spotify:track:6tNQ70jh4OwmPGpYy6R2o9,0.219
3,Like That,Future,0.007090,https://api.spotify.com/v1/audio-analysis/2tud...,0.814,267707,0.676,2tudvzsrR56uom6smgOcSf,0.000013,11,0.1190,-4.670,0,0.2310,162.012,4,https://api.spotify.com/v1/tracks/2tudvzsrR56u...,audio_features,spotify:track:2tudvzsrR56uom6smgOcSf,0.312
4,i like the way you kiss me,Artemas,0.000938,https://api.spotify.com/v1/audio-analysis/2Gxr...,0.599,142515,0.946,2GxrNKugF82CnoRFbQfzPf,0.010600,11,0.0826,-4.263,1,0.0447,151.647,4,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,audio_features,spotify:track:2GxrNKugF82CnoRFbQfzPf,0.747


In [22]:
#Exporting the data
track_data.to_csv('track_data.csv', index=False)